In [2]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Cloning https://github.com/huggingface/transformers.git to c:\users\alvin\appdata\local\temp\pip-req-build-y0bs4hhx
  Resolved https://github.com/huggingface/transformers.git to commit ec905f3a761bda50cb1d9402eb4e655a1d392732
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\alvin\AppData\Local\Temp\pip-req-build-y0bs4hhx'


In [3]:
import librosa

audio_file_path = 'PrelimNoTranscription.mp3'
audio_array, sampling_rate = librosa.load(audio_file_path, sr=16000)

In [4]:
audio_data = {
    "array": audio_array,
    "sampling_rate": sampling_rate
}

audio_data, len(audio_data['array'])


({'array': array([ 1.2109997e-05,  2.7982878e-05,  2.4542742e-05, ...,
         -6.6963089e-06, -4.1230073e-06,  1.9476836e-06], dtype=float32),
  'sampling_rate': 16000},
 20367035)

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

model_id = "openai/whisper-small"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)


cuda:0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    return_timestamps=True,  # Set this to True
    torch_dtype=torch_dtype,
    device=device,
)

In [7]:
# ## whisper-small
# input_features = processor(audio_data["array"], sampling_rate=audio_data["sampling_rate"], return_tensors="pt").input_features
# # generate token ids
# predicted_ids = model.generate(input_features)
# # decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
# transcription


## pipeline method
transcription = pipe(audio_data)

c:\Users\alvin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Users\alvin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
The attention mask is not set and cannot be inferre

In [8]:
import json

exDict = {'exDict': transcription}

with open('PrelimTranscript_test.txt', 'w') as file:
     file.write(json.dumps(exDict))


In [9]:
print(transcription)

{'text': " obvious is what is a cryptocurrency? So this word was kind of invented 10 years ago when, I don't know how many of you know the origin story of where Bitcoin came from, but basically a pseudonym on the internet dropped a paper and some open source code in a forum on an email list and said, hey, I have this idea for this thing called Bitcoin. It's kind of like electronic cash. Here's how I think it could work. And here is some code if you want to run it and become part of this peer-to-peer network. We don't know who this person is. This person is basically virtually disappeared from the internet and from the world. But it's created something that has captured so many people's imaginations and has sort of, depending on how you measure it, created billions and billions of dollars of economic value and sort of inspired a lot of people to think about how to use this technology to solve a myriad of different problems, not just electronic payments. So cryptocurrencies and the techn

In [13]:
for chunk in transcription["chunks"]:
    time = chunk["timestamp"]
    print(time)

(0.0, 2.8)
(2.8, 7.88)
(7.88, 10.24)
(10.24, 13.68)
(13.68, 18.48)
(18.48, 20.36)
(20.36, 22.92)
(22.92, 25.0)
(25.0, 26.28)
(26.28, 28.24)
(0.0, 2.34)
(2.34, 3.88)
(3.88, 6.4)
(6.4, 9.04)
(9.04, 12.12)
(12.12, 15.44)
(15.44, 16.68)
(16.68, 20.3)
(20.3, 23.84)
(23.84, 25.84)
(25.84, 27.36)
(27.36, 29.72)
(0.0, 4.04)
(4.04, 9.32)
(9.32, 13.36)
(13.36, 16.64)
(16.64, 22.52)
(22.52, 26.08)
(26.08, 28.72)
(0.0, 2.88)
(2.88, 6.16)
(6.16, 9.32)
(9.32, 11.4)
(11.4, 15.52)
(15.52, 17.76)
(17.76, 18.76)
(18.76, 21.68)
(21.68, 23.48)
(23.48, 26.24)
(0.0, 3.32)
(3.32, 5.48)
(5.48, 8.0)
(8.0, 11.68)
(11.68, 13.56)
(13.56, 17.96)
(17.96, 22.0)
(22.0, 22.8)
(22.8, 26.52)
(26.52, 29.12)
(0.0, 3.4)
(3.4, 6.88)
(6.88, 10.12)
(10.12, 11.64)
(11.64, 14.16)
(14.16, 17.6)
(17.6, 19.44)
(19.44, 22.0)
(22.0, 25.0)
(25.0, 27.24)
(27.24, 29.76)
(0.0, 7.48)
(7.48, 11.92)
(11.92, 18.96)
(18.96, 27.72)
(0.0, 7.64)
(7.64, 10.16)
(10.16, 15.12)
(15.12, 19.72)
(19.72, 25.04)
(25.04, 26.04)
(0.0, 5.72)
(5.72, 8.8)
(8